In [2]:
import os
import numpy as np
import cv2
import glob
from sklearn.model_selection import train_test_split


In [3]:
# Define directories
vv_dir = '/home/maelh/Downloads/NASA_database/vv/'
vh_dir = '/home/maelh/Downloads/NASA_database/vh/'
water_body_dir = '/home/maelh/Downloads/NASA_database/water_body_label/'
flood_label_dir = '/home/maelh/Downloads/NASA_database/flood_label/'
preprocessed_dir = '../preprocessed_data/'


In [4]:


def load_images_from_folder(folder, label_folder):
    # Initialiser les tableaux NumPy avec la taille requise
    num_images = len(glob.glob(os.path.join(folder, '*.png')))
    images = np.zeros((num_images, 256, 256), dtype=np.uint8)
    labels = np.zeros((num_images, 256, 256), dtype=np.uint8)

    for i, filename in enumerate(glob.glob(os.path.join(folder, '*.png'))):
        # Charger l'image en uint8
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Erreur de chargement de l'image: {filename}")
            continue  # Passer à l'image suivante
        
        img = cv2.resize(img, (256, 256))  # Redimensionner
        images[i] = img  # Remplir le tableau NumPy
        
        # Charger le label correspondant
        label_name = os.path.basename(filename).replace('_vv.png', '.png')
        label_path = os.path.join(label_folder, label_name)
        label_img = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
        if label_img is None:
            print(f"Erreur de chargement du label: {label_path}")
            continue  # Passer au label suivant
        
        label_img = cv2.resize(label_img, (256, 256))
        binary_label = (label_img > 0).astype(np.uint8)
        labels[i] = binary_label  # Remplir le tableau NumPy

    # Retirer les images et labels qui n'ont pas pu être chargés
    valid_indices = np.where(images.sum(axis=(1, 2)) > 0)[0]
    return images[valid_indices], labels[valid_indices]

# Exemple d'utilisation



In [5]:
images, labels = load_images_from_folder(vv_dir,water_body_dir)

X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models

def unet_model_with_water_body(input_shape=(256, 256, 1)):
    """
    Build a U-Net model that takes in VV, VH polarization images and 
    the water body label as input to predict the flooded areas.
    
    Input: (VV, VH, Water Body Label) -> 3 channels
    Output: Flooded area segmentation -> 1 channel
    """

    inputs = layers.Input(shape=input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    # Decoder
    u1 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(p2)
    u1 = layers.concatenate([u1, c2])
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c3)

    u2 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c3)
    u2 = layers.concatenate([u2, c1])
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c4)

    # Output layer (flooded area prediction)
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c4)

    # Compile the model
    model = models.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Example usage:
# The input shape is (256, 256, 3) where we have 3 channels (VV, VH, Water Body Label)
model = unet_model_with_water_body()
model.summary()


2024-10-09 10:16:00.987480: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-09 10:16:00.997461: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-09 10:16:01.096740: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-09 10:16:01.180743: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-09 10:16:01.261943: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │        640 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     36,928 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │    147,584 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 128, 128,  │     65,664 │ max_pooling2d_1[… │
│ (Conv2DTranspose)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 128, 128,  │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 256)              │            │ conv2d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 128, 128,  │    295,040 │ concatenate[0][0] │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 128, 128,  │    147,584 │ conv2d_4[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_1  │ (None, 256, 256,  │     32,832 │ conv2d_5[0][0]    │
│ (Conv2DTranspose)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 256, 256,  │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 128)              │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 256, 256,  │     73,792 │ concatenate_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 256, 256,  │     36,928 │ conv2d_6[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 256, 256,  │         65 │ conv2d_7[0][0]    │
│                     │ 1)                │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 910,913 (3.47 MB)

 Trainable params: 910,913 (3.47 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
def iou_metric(y_true, y_pred, threshold=0.5):
    """
    Calculate Intersection over Union (IoU) between ground truth (y_true) and predicted (y_pred) masks.
    
    Arguments:
    y_true -- Ground truth binary mask (e.g., flood label).
    y_pred -- Predicted binary mask (output from the model).
    threshold -- Threshold to convert the predicted probabilities into binary output.
    
    Returns:
    IoU score.
    """

    # Convert predicted probabilities to binary (0 or 1) based on the threshold
    y_pred_binary = (y_pred > threshold).astype(np.uint8)

    # Compute intersection and union
    intersection = np.logical_and(y_true, y_pred_binary)
    union = np.logical_or(y_true, y_pred_binary)

    # Calculate IoU
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

In [8]:
# Train the U-Net model
model.fit(X_train, y_train, epochs=3, batch_size=4, validation_data=(X_val, y_val))

Epoch 1/3
285/285 ━━━━━━━━━━━━━━━━━━━━ 1513s 5s/step - accuracy: 0.9821 - loss: 0.0620 - val_accuracy: 0.9783 - val_loss: 0.0758
Epoch 2/3
285/285 ━━━━━━━━━━━━━━━━━━━━ 1162s 4s/step - accuracy: 0.9803 - loss: 0.0902 - val_accuracy: 0.9794 - val_loss: 0.0619
Epoch 3/3
285/285 ━━━━━━━━━━━━━━━━━━━━ 1221s 4s/step - accuracy: 0.9832 - loss: 0.0640 - val_accuracy: 0.9785 - val_loss: 0.0620


In [9]:

##### PREDICTION DU MODELE #####
# After training, make predictions on a subset of the validation set (3 samples)
y_pred = model.predict(X_val[:10])
print(y_pred.shape)

# Calculate the IoU metric for each prediction
iou_scores = []
for i in range(10):
    intersection = np.logical_and(y_val[i], y_pred[i])
    union = np.logical_or(y_val[i], y_pred[i])
    if np.sum(union) == 0:
        iou_score = 0.0
    else:
        iou_score = np.sum(intersection) / np.sum(union)
    iou_scores.append(iou_score)

print("Average IoU score for 3 samples:", np.mean(iou_scores))

NameError: name 'model' is not defined

In [14]:
##### IMPRIME LES IMAGES EN PNG ####
import imageio

# Ensure the output directory exists
output_dir = '../predicted_images'
ref_dir = '../predicted_images/reference'
os.makedirs(output_dir, exist_ok=True)

# Transform the 3 predicted images into PNG files
for i in range(10):
    # Rescale the image to 0-255 and convert to uint8
    # img = (y_pred[i, :, :, 0] * 255).astype(np.uint8)
    # # Save the image as a PNG file
    # imageio.imwrite(os.path.join(output_dir, f'predicted_image_{i}.png'), img)
    #save the corresponding reference image
    ref = (y_val[i, :, :] * 255).astype(np.uint8)
    sat_im = (X_val[i, :, :] * 255).astype(np.uint8)
    # Save the image as a PNG file
    imageio.imwrite(os.path.join(ref_dir, f'predicted_image_water{i}.png'), ref)
    imageio.imwrite(os.path.join(ref_dir, f'predicted_image_vv{i}.png'), sat_im)


In [7]:
X_val

array([[[114, 129, 135, ..., 104, 102, 102],
        [116, 133, 138, ..., 108, 101, 101],
        [106, 120, 130, ..., 108, 104, 104],
        ...,
        [106, 110, 120, ..., 104, 114, 114],
        [121, 123, 126, ..., 104, 114, 114],
        [135, 134, 131, ..., 116, 124, 124]],

       [[  9,   4,   0, ...,  58,  81, 109],
        [  5,   6,   5, ...,  18,  39,  91],
        [  1,   6,  17, ...,   0,   6,  56],
        ...,
        [139, 134, 127, ..., 131, 120, 123],
        [137, 137, 134, ..., 130, 122, 123],
        [139, 140, 139, ..., 122, 124, 124]],

       [[157, 160, 160, ..., 147, 147, 150],
        [150, 149, 155, ..., 150, 154, 155],
        [153, 154, 158, ..., 151, 155, 160],
        ...,
        [151, 136, 129, ..., 139, 157, 169],
        [152, 137, 129, ..., 160, 174, 189],
        [161, 143, 133, ..., 166, 189, 198]],

       ...,

       [[ 56,  72,  67, ..., 124, 129, 129],
        [ 45,  61,  68, ..., 126, 133, 127],
        [ 53,  56,  56, ..., 127, 130, 119